In [3]:
!pip install plotly[express]
!pip install kaggle

In [24]:
import pandas as pd
import numpy as np
import plotly.express as px 
import matplotlib.pyplot as plt
import os
import zipfile
from kaggle.api.kaggle_api_extended import KaggleApi
from scipy import stats

In [5]:
%run "../src/descarga remota.py"

Iniciando descarga del dataset: titanic
Descarga completada. Archivos guardados en: ../data
Descomprimiendo titanic.zip...
Archivo titanic.zip descomprimido y eliminado.

¡Proceso de descarga y extracción finalizado con éxito!


In [10]:
df_test = pd.read_csv("../data/test.csv")
df_train = pd.read_csv("../data/train.csv")

In [11]:
print("DataFrame de test.csv: ")
display(df_test.head(5))

print("DataFrame de train.csv: ")
display(df_train.head(5))

DataFrame de test.csv: 


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


DataFrame de train.csv: 


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# Descripción variables 
## (que representa cada una y un breve analisis estadistico basico)

Basta con observar el apartado **Data Dictionary** en la pagina de [kaggle](https://www.kaggle.com/competitions/titanic/data) y es posible observar el significado de cada variable.

## Descripción de Variables del Conjunto de Datos

| Variable | Definición | Key (Clave) |
| :--- | :--- | :--- |
| **survival** | Supervivencia | 0 = No, 1 = Sí |
| **pclass** | Clase del billete/boleto | 1 = 1ra, 2 = 2da, 3 = 3ra |
| **sex** | Sexo | |
| **Age** | Edad en años | |
| **sibsp** | # de hermanos / cónyuges a bordo del Titanic | |
| **parch** | # de padres / niños a bordo del Titanic | |
| **ticket** | Número del billete/boleto | |
| **fare** | Tarifa del pasajero | |
| **cabin** | Número de cabina | |
| **embarked** | Puerto de Embarque | C = Cherbourg, Q = Queenstown, S = Southampton |

#### Ademas se hacen otras salvedades sobre las variables

##### **pclass:** Un proxy para el estatus socioeconómico (SES)
* **1ra:** Alta (Upper)
* **2da:** Media (Middle)
* **3ra:** Baja (Lower)

---

##### **age:** Edad
* La edad es **fraccionaria** si es menor de 1 año.
* Si la edad es **estimada**, se presenta en la forma de *xx.5* (ej. 29.5).

---

##### **sibsp:** Relaciones familiares (Hermanos/Cónyuges)
El conjunto de datos define las relaciones familiares de esta manera...
* **Hermano/a:** hermano, hermana, hermanastro, hermanastra
* **Cónyuge:** marido, mujer
    > *(Se ignoraron a las amantes y prometidos/as.)*

---

##### **parch:** Relaciones familiares (Padres/Hijos)
El conjunto de datos define las relaciones familiares de esta manera...
* **Padre:** madre, padre
* **Hijo/a:** hija, hijo, hijastra, hijastro
    > **Nota:** Algunos niños viajaron solo con una niñera, por lo tanto, **parch=0** para ellos.

> Note que la variable **Survived** solo esta en **train**, **no** en **test** ya que es la **target variable**

### Información Basica de los DataFrame

In [12]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.1+ KB


> Aprecie que **Cabin** tiene tan solo 91 datos no nulos, un problema con el que se lideara mas adelante.

> hay 4  variables int, 5 object y 2 float

### Limpieza de datos

In [13]:
df_test.isnull().any()

PassengerId    False
Pclass         False
Name           False
Sex            False
Age             True
SibSp          False
Parch          False
Ticket         False
Fare            True
Cabin           True
Embarked       False
dtype: bool

In [14]:
df_train.isnull().any()

PassengerId    False
Survived       False
Pclass         False
Name           False
Sex            False
Age             True
SibSp          False
Parch          False
Ticket         False
Fare           False
Cabin           True
Embarked        True
dtype: bool

#### Limpieza df_train

Note que las columnas del Data Frame **df_train** las cuales son **Age**, **Cabin**, **Embarked**; poseen datos nulos, se procederá a analizar la mejor estrategia para lidiar con los valores nulos.

##### Limpieza columna **Age**

Se revisa cuantos datos nulos existen en la columna **Age** y se contrasta con el total de los datos.

In [18]:
df_train["Age"].isnull().sum()

177

In [19]:
len(df_train)

891

Se revisa cuanto porcentaje de datos nulos existe.

In [20]:
porcentaje_nulos = df_train["Age"].isnull().mean() * 100
print(f"El porcentaje de nulos en la columna'Age' es: {porcentaje_nulos:.2f}%")

El porcentaje de nulos en la columna'Age' es: 19.87%


Luego de esto, evaluamos la normalidad de la variable **Age**, para esto creamos el histograma de la columna.

In [23]:
n_obs = len(df_train)
# Calcular las posiciones para los cuantiles
# np.linspace(0.01, 0.99, n_obs) crea una serie de probabilidades
quantiles_prob = np.linspace(1, n_obs, n_obs) / (n_obs + 1)

# Calcular los cuantiles teóricos de la distribución Normal estándar (Z-score)
teoretical_quantiles = stats.norm.ppf(quantiles_prob)

# --- 3. Crear el DataFrame para el Gráfico ---
qq_df = pd.DataFrame({
    'Theoretical Quantiles': teoretical_quantiles,
    'Sample Quantiles (Age)': data.values
})

# --- 4. Crear el Gráfico Q-Q (Scatter Plot) con Plotly ---
fig = px.scatter(
    qq_df,
    x='Theoretical Quantiles',
    y='Sample Quantiles (Age)',
    title='Gráfico Q-Q (Cuantil-Cuantil) de Age vs. Distribución Normal',
    labels={'Theoretical Quantiles': 'Cuantiles Teóricos (Normal Z)',
            'Sample Quantiles (Age)': 'Edad Observada (Cuantil de Muestra)'},
    height=600
)

# --- 5. Añadir la Línea de Referencia 45° (Esperada) ---
# Se necesita una línea recta para comparar. La trazamos entre los valores mín/máx.
min_val = min(qq_df['Theoretical Quantiles'].min(), qq_df['Sample Quantiles (Age)'].min())
max_val = max(qq_df['Theoretical Quantiles'].max(), qq_df['Sample Quantiles (Age)'].max())

fig.add_trace(
    go.Scatter(
        x=[min_val, max_val], 
        y=[min_val, max_val], 
        mode='lines',
        name='Línea de Referencia',
        line=dict(color='red', dash='dash')
    )
)

fig.update_layout(showlegend=True)
fig.show()

NameError: name 'stats' is not defined

### Analisis estadistico basico para las variables:

In [ ]:
columnas = df_train.columns   #se usa el de todas las col

: 

: 

: 

: 

: 

: 

: 

In [ ]:
def analizar(data, cols):
    for col in cols:
        try:
            print(f"columna: {col}")
            display(data[col].describe())
            print("\n\n\n")
        except Exception as e:
            print("columna no esta\n\n\n")

: 

: 

: 

: 

: 

: 

: 

In [ ]:
# analisis general para df_test
analizar(df_test, columnas)

columna: PassengerId


count     418.000000
mean     1100.500000
std       120.810458
min       892.000000
25%       996.250000
50%      1100.500000
75%      1204.750000
max      1309.000000
Name: PassengerId, dtype: float64





columna: Survived
columna no esta



columna: Pclass


count    418.000000
mean       2.265550
std        0.841838
min        1.000000
25%        1.000000
50%        3.000000
75%        3.000000
max        3.000000
Name: Pclass, dtype: float64





columna: Name


...

: 

: 

: 

: 

: 

In [ ]:
# analisis general para df_train
analizar(df_train, columnas)

columna: PassengerId


count    891.000000
mean     446.000000
std      257.353842
min        1.000000
25%      223.500000
50%      446.000000
75%      668.500000
max      891.000000
Name: PassengerId, dtype: float64





columna: Survived


count    891.000000
mean       0.383838
std        0.486592
min        0.000000
25%        0.000000
50%        0.000000
75%        1.000000
max        1.000000
Name: Survived, dtype: float64





columna: Pclass


...

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 

: 